#Tarefa 5 - Regressão Linear e logística

## Nesta tarefa, você deve carregar um dataset sobre as vendas de cadeirinhas infantis, construir modelo de Regressão com os algoritmos vistos em aula e predizer o valor das vendas.

Dica: Para toda a tarefa, além da biblioteca pandas e numpy, você pode querer explorar funções da biblioteca sklearn.linear_model (em particular os pacotes LinearRegression e LogisticRegression).
Além disso, você vai precisar usar funções de pré-processamento e de pós-procesamento (das bibliotecas sklearn.preprocessing, sklearn.model_selection e sklearn.metrics)


###Importe os pacotes e carregue o arquivo com os dados
O dataset a ser utilizado encontra-se no arquivo casseats.csv, disponível no EAD.


Este dataset contém dados sobre as vendas de cadeirinhas infantis em 400 lojas diferentes, descritas pelos seguintes atributos/variáveis:
* Sales - Vendas unitárias (em milhares) em cada local
* CompPrice - Preço cobrado pelo concorrente em cada local
* Income - Nível de renda da comunidade (em milhares de dólares)
* Advertising - Orçamento de publicidade local para a empresa em cada local (em milhares de dólares)
* Population - Tamanho da população na região (em milhares)
* Price - Preço que a empresa cobra por assentos de carro em cada local
* ShelveLoc - Um fator com níveis Ruim, Bom e Médio que indica a qualidade da localização das estantes para as cadeirinhas em cada local
* Age - Idade média da população local
* Education - Nível de escolaridade em cada localidade
* Urban - Um fator com níveis Não e Sim para indicar se a loja está localizada em área urbana ou rural
* US - Um fator com níveis Não e Sim para indicar se a loja é nos EUA ou não
* High - Um fator com níveis Não e Sim para indicar se a compra é alta ou não

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OrdinalEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, roc_curve, auc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carregar a base de dados
df = pd.read_csv('Carseats.csv', sep=';', index_col=0)

df.head()

In [ ]:
df.describe()

In [ ]:
print(df['Urban'].unique())
print(df['US'].unique())
print(df['High'].unique())
print(df['ShelveLoc'].unique())

### Pré-processe a base de dados

Dica: avalie a necessidade de converter os tipos das variáveis, normalizar os dados, ...

In [ ]:
le_urban = LabelEncoder()
le_us = LabelEncoder()
le_high = LabelEncoder()
oe_shelve_loc = OrdinalEncoder(categories=[['Bad', 'Medium', 'Good']])
scaler = MinMaxScaler()

def preprocessing(df: pd.DataFrame):
    if not hasattr(le_urban, 'classes_'):
        df['Urban'] = le_urban.fit_transform(df['Urban'])
    else:
        df['Urban'] = le_urban.transform(df['Urban'])
        
    if not hasattr(le_us, 'classes_'):
        df['US'] = le_us.fit_transform(df['US'])
    else:
        df['US'] = le_us.transform(df['US'])
        
    if not hasattr(le_high, 'classes_'):
        df['High'] = le_high.fit_transform(df['High'])
    else:
        df['High'] = le_high.transform(df['High'])
    
    if not hasattr(oe_shelve_loc, 'categories_'):
        df['ShelveLoc'] = oe_shelve_loc.fit_transform(df[['ShelveLoc']])
    else:
        df['ShelveLoc'] = oe_shelve_loc.transform(df[['ShelveLoc']])
    
    df['Sales'] = df['Sales'].apply(lambda x: x.replace(',', '.'))
    
    if not hasattr(scaler, 'mean_'):
        df[df.columns] = scaler.fit_transform(df[df.columns])
    else:
        df[df.columns] = scaler.transform(df[df.columns])
    return df

df = preprocessing(df)

df.head()

### Crie os conjuntos de treinamento e de teste

Atenção: Selecione aleatoriamente e sem reposição (para que não se repitam) 320 registros para o conjunto de treinamento. As 80 observações restantes serão usadas para o conjunto de teste. Fixe a semente de geração de dados aleatórios em 456.

In [ ]:
# Dividir os dados em conjunto de treino e teste
df_linear = df.drop('High', axis=1)

X = df_linear.drop('Sales', axis=1)
y = df_linear['Sales']

np.random.seed(456)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)


In [ ]:
# visualizar os dados

plt.figure(figsize=(8, 6))
sns.histplot(df['Sales'], kde=True, bins=20, color='blue')
plt.title('Distribuição das Vendas (Sales)', fontsize=16)
plt.xlabel('Vendas (em milhares)', fontsize=14)
plt.ylabel('Frequência', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Price', y='Sales', data=df, color='green')
plt.title('Relação entre Preço da Empresa e Vendas', fontsize=16)
plt.xlabel('Preço da Empresa (em milhares)', fontsize=14)
plt.ylabel('Vendas (em milhares)', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='ShelveLoc', y='Sales', data=df, hue='ShelveLoc', palette='Set2')
plt.title('Vendas por Qualidade da Localização das Estantes', fontsize=16)
plt.xlabel('Qualidade da Estante', fontsize=14)
plt.ylabel('Vendas (em milhares)', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Income', y='Sales', data=df, hue='Urban', palette='coolwarm')
plt.title('Relação entre Renda da Comunidade e Vendas', fontsize=16)
plt.xlabel('Renda', fontsize=14)
plt.ylabel('Vendas', fontsize=14)
plt.legend(title='Área Urbana')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlação entre Variáveis Numéricas', fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='Urban', y='Sales', data=df, hue='Urban', palette='pastel')
plt.title('Vendas por Localização Urbana ou Rural', fontsize=16)
plt.xlabel('Localização Urbana', fontsize=14)
plt.ylabel('Vendas', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='US', y='Sales', data=df, hue='US', palette='muted')
plt.title('Vendas entre Lojas nos EUA e Fora', fontsize=16)
plt.xlabel('Loja nos EUA', fontsize=14)
plt.ylabel('Vendas', fontsize=14)
plt.show()

### Construa um modelo de Regressão Linear Múltipla.

Utilizando o modelo, faça a predição do atributo Sales no conjunto teste

Atenção: o atributo High deve ser eliminado para este processo

In [ ]:
regressao_linear = LinearRegression()

regressao_linear.fit(X_train, y_train)
y_pred = regressao_linear.predict(X_test)

### Pós-processamento: Avalie cada modelo de regressão

Calcule as medidas de desempenho vistas em aula (erro quadrático médio, R2)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Performance on the Test Set:")
print(f"MSE: {mse:.4%}")
print(f"R2 Score: {r2:.4%}")

### Agora, construa um modelo de Regressão Logística

Faça a predição do atributo High no conjunto teste, e avalie utilizando métricas adequadas

Atenção: o atributo Sales deve ser eliminado para este processo

In [ ]:
# Didivir dados de treino e teste

df_linear = df.drop('Sales', axis=1)

X = df_linear.drop('High', axis=1)
y = df_linear['High']

np.random.seed(456)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)


# Treinar o modelo de Regressão Logística

regressao_logistica = LogisticRegression()

regressao_logistica.fit(X_train, y_train)

# Fazer predições com o conjunto de teste

y_pred = regressao_logistica.predict(X_test)

y_prob = regressao_logistica.predict_proba(X_test)[:, 1]

# Calcular métricas de desempenho

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

print("Performance on the Test Set:")
print(f"Test Accuracy: {accuracy:.4%}")
print(f"Test Precision: {precision:.4%}")
print(f"Test Recall: {recall:.4%}")
print(f"Test F1 Score: {f1:.4%}")
print(f"Test ROC AUC Score: {roc_auc:.4%}")